In [ ]:
#I run Task4 on colab, so I read data from google drive!
from google.colab import drive
drive.mount('/content/drive/')

!ls /content/drive/MyDrive/



Mounted at /content/drive/
'Colab Notebooks'   fake_job_postings.csv   sample_data.csv


In [ ]:
!pip install transformers datasets torch scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
＃Task4-A:fine-tuning DistilBERT

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
from sklearn.model_selection import train_test_split

# Read the dataset
# If you need to run on local computer!
#import os
#data_file = os.path.join("data", "fake_job_postings.csv")
#df = pd.read_csv(data_file)

df = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv')

# Task 1: Predicting Education Level
non_fraud_df = df[df['fraudulent'] == 0]
education_levels = ['Master\'s Degree', 'Bachelor\'s Degree', 'High School or equivalent']
education_df = non_fraud_df[non_fraud_df['required_education'].isin(education_levels)]

# Split the dataset into training and validation sets
train_dataset_task1, val_dataset_task1 = train_test_split(education_df, test_size=0.2)

# Task 2: Predict whether it is a fake job
fraud_df = df[['description', 'fraudulent']]
train_dataset_task2, val_dataset_task2 = train_test_split(fraud_df, test_size=0.2)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Prepare preprocessing functions for Task 1: Education Level Prediction
def preprocess_data(examples):
    description = examples['description'] if isinstance(examples['description'], str) else ''
    required_education = examples['required_education'] if pd.notna(examples['required_education']) else 'Unknown'

    tokenized_inputs = tokenizer(description, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = education_levels.index(required_education) if required_education in education_levels else -1
    return tokenized_inputs

# Prepare preprocessing functions for Task 2: Fake job detection
def preprocess_data_fraud(examples):
    description = examples['description'] if isinstance(examples['description'], str) else ''
    tokenized_inputs = tokenizer(description, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['fraudulent']
    return tokenized_inputs

# Process the dataset
train_dataset_task1 = train_dataset_task1.apply(preprocess_data, axis=1).tolist()
val_dataset_task1 = val_dataset_task1.apply(preprocess_data, axis=1).tolist()
train_dataset_task2 = train_dataset_task2.apply(preprocess_data_fraud, axis=1).tolist()
val_dataset_task2 = val_dataset_task2.apply(preprocess_data_fraud, axis=1).tolist()

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")

# Function to calculate the indicator
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    return accuracy

# Task 1: Education level prediction
model_education = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(education_levels))
training_args_task1 = TrainingArguments(
    output_dir='./results_education',
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)
trainer_education = Trainer(
    model=model_education,
    args=training_args_task1,
    train_dataset=train_dataset_task1,
    eval_dataset=val_dataset_task1,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Task 2: Fake job detection
model_fraud = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
training_args_task2 = TrainingArguments(
    output_dir='./results_fraud',
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)
trainer_fraud = Trainer(
    model=model_fraud,
    args=training_args_task2,
    train_dataset=train_dataset_task2,
    eval_dataset=val_dataset_task2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and evaluate Task 1 (education level prediction)
trainer_education.train()
trainer_education.evaluate()

# Train and evaluate Task 2 (Fake Job Detection)
trainer_fraud.train()
trainer_fraud.evaluate()

# Save the model
trainer_education.save_model("./model_education")
trainer_fraud.save_model("./model_fraud")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.

Step,Training Loss,Validation Loss,Accuracy
500,0.345400,0.361247,0.905995
1000,0.197600,0.363812,0.914850


Step,Training Loss,Validation Loss,Accuracy
500,0.153100,0.110242,0.973993
1000,0.101900,0.075821,0.980145
1500,0.071200,0.069862,0.979586
2000,0.043200,0.078459,0.983501
2500,0.021700,0.082315,0.984899


In [ ]:
＃Task4-A:Orignal DistilBERT

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
from sklearn.model_selection import train_test_split

# Read the dataset
# change if you want
df = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv')

# Task 1: Predicting Education Level
non_fraud_df = df[df['fraudulent'] == 0]
education_levels = ['Master\'s Degree', 'Bachelor\'s Degree', 'High School or equivalent']
education_df = non_fraud_df[non_fraud_df['required_education'].isin(education_levels)]

# Split the dataset into training and validation sets
train_dataset_task1, val_dataset_task1 = train_test_split(education_df, test_size=0.2)

# Task 2: Predict whether it is a fake job
fraud_df = df[['description', 'fraudulent']]
train_dataset_task2, val_dataset_task2 = train_test_split(fraud_df, test_size=0.2)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Prepare preprocessing functions for Task 1: Education Level Prediction
def preprocess_data(examples):
    description = examples['description'] if isinstance(examples['description'], str) else ''
    required_education = examples['required_education'] if pd.notna(examples['required_education']) else 'Unknown'

    tokenized_inputs = tokenizer(description, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = education_levels.index(required_education) if required_education in education_levels else -1
    return tokenized_inputs

# Prepare preprocessing functions for Task 2: Fake job detection
def preprocess_data_fraud(examples):
    description = examples['description'] if isinstance(examples['description'], str) else ''
    tokenized_inputs = tokenizer(description, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['fraudulent']
    return tokenized_inputs

# Process the dataset
train_dataset_task1 = train_dataset_task1.apply(preprocess_data, axis=1).tolist()
val_dataset_task1 = val_dataset_task1.apply(preprocess_data, axis=1).tolist()
train_dataset_task2 = train_dataset_task2.apply(preprocess_data_fraud, axis=1).tolist()
val_dataset_task2 = val_dataset_task2.apply(preprocess_data_fraud, axis=1).tolist()

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")

# Function to calculate the indicator
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    return accuracy

# Task 1: Education level prediction (untrained model)
model_education = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(education_levels))
training_args_task1 = TrainingArguments(
    output_dir='./results_education',
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0,  # No training
    weight_decay=0.01
)
trainer_education = Trainer(
    model=model_education,
    args=training_args_task1,
    train_dataset=train_dataset_task1,
    eval_dataset=val_dataset_task1,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Task 2: Fake Job Detection (Untrained Model)
model_fraud = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
training_args_task2 = TrainingArguments(
    output_dir='./results_fraud',
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0, # No training
    weight_decay=0.01
)
trainer_fraud = Trainer(
    model=model_fraud,
    args=training_args_task2,
    train_dataset=train_dataset_task2,
    eval_dataset=val_dataset_task2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate Task 1 (Education Level Prediction)
eval_results_education = trainer_education.evaluate()
print("Education Level Prediction Evaluation Results:", eval_results_education)

# Evaluate Task 2 (Fraudulent Job Detection)
eval_results_fraud = trainer_fraud.evaluate()
print("Fraudulent Job Detection Evaluation Results:", eval_results_fraud)

# Save the model (although not trained, it can still be saved)
trainer_education.save_model("./model_education")
trainer_fraud.save_model("./model_fraud")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForS

Education Level Prediction Evaluation Results: {'eval_loss': 1.1107063293457031, 'eval_model_preparation_time': 0.0028, 'eval_accuracy': 0.138283378746594, 'eval_runtime': 24.0182, 'eval_samples_per_second': 61.12, 'eval_steps_per_second': 3.83}


Fraudulent Job Detection Evaluation Results: {'eval_loss': 0.6959530115127563, 'eval_model_preparation_time': 0.0038, 'eval_accuracy': 0.48126398210290827, 'eval_runtime': 54.9838, 'eval_samples_per_second': 65.037, 'eval_steps_per_second': 4.074}


In [ ]:
#Task4-B: GPT-4o-Mini

In [ ]:
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existin

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# I have api key, However I can not provide for you!
# Set up OpenAI API key
OPENAI_API_KEY = ""

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")


# Example: Categorize a single job description
def classify_job_description(job_description):
    input_text = f"Predict the required education level for the following job description(only show the degree label, it may be label-Master’s Degree==2, Bachelor’s Degree==1 or High School or equivalent==0 ”): {job_description}"
    response = llm.invoke(input_text)
    return response.content.strip() if hasattr(response, 'content') else response.strip()

# Test classification function
job_description = "As part of the sales team, you will work with clients to develop and execute strategies to help them leverage GIS technology for business goals."
predicted_education = classify_job_description(job_description)
print(f"The predicted education requirement is: {predicted_education}")

The predicted education requirement is: 1


In [7]:
import pandas as pd
import numpy as np
import time


# Set OpenAI API key
OPENAI_API_KEY = ""

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")

# Read dataset
# change if you want
df = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv')

# Filter out non-fraudulent job postings
df = df[df['fraudulent'] == 0]

# Filter based on required education levels
education_levels = ["Bachelor's Degree", "Master's Degree", 'High School or equivalent']
df = df[df['required_education'].isin(education_levels)]

# Map education levels: Master's Degree==2, Bachelor's Degree==1, High School or equivalent==0
education_mapping = {
    "Master's Degree": 2,
    "Bachelor's Degree": 1,
    "High School or equivalent": 0
}
y = df['required_education'].map(education_mapping)
print(f"Length of y: {len(y)}")
Xr = df['description']
print(f"Length of Xr: {len(Xr)}")

# Define function to classify job descriptions
def classify_job_description(job_description):
    input_text = f"Predict the required education level for the following job description (only show the degree label, it may be: Master's Degree==2, Bachelor's Degree==1, or High School or equivalent==0): {job_description}"
    response = llm.predict(input_text)
    return response.strip()

# Define batch processing function
def classify_in_batches(Xr, batch_size=200):
    predictions = []

    # Loop through the dataset in batches
    for i in range(0, len(Xr), batch_size):
        batch_descriptions = Xr[i:i+batch_size]
        print(f"Processing batch {i // batch_size + 1}/{(len(Xr) // batch_size) + 1}")

        # Classify each job description in the batch
        for job_description in batch_descriptions:
            predicted_label = classify_job_description(job_description)
            try:
                predictions.append(int(predicted_label))  # Convert the prediction to an integer
            except ValueError:
                predictions.append(-1)  # If the result can't be classified, assign -1

        # Optional: Add a delay between batches to avoid API rate limits
        time.sleep(1)

    return predictions

# Process the job descriptions in batches
batch_size = 200  # You can adjust the batch size as needed
predictions = classify_in_batches(Xr, batch_size=batch_size)

# Define function to calculate accuracy
def calculate_accuracy(predictions, true_labels):
    # Ignore unclassified results (-1) and calculate accuracy
    valid_predictions = [pred for pred in predictions if pred != -1]
    valid_true_labels = [label for pred, label in zip(predictions, true_labels) if pred != -1]
    accuracy = np.mean([pred == true_label for pred, true_label in zip(valid_predictions, valid_true_labels)])
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(predictions, y)
print(f"Classification accuracy: {accuracy * 100:.2f}%")


Length of y: 7340
Length of Xr: 7340
Processing batch 1/37
Processing batch 2/37
Processing batch 3/37
Processing batch 4/37
Processing batch 5/37
Processing batch 6/37
Processing batch 7/37
Processing batch 8/37
Processing batch 9/37
Processing batch 10/37
Processing batch 11/37
Processing batch 12/37
Processing batch 13/37
Processing batch 14/37
Processing batch 15/37
Processing batch 16/37
Processing batch 17/37
Processing batch 18/37
Processing batch 19/37
Processing batch 20/37


KeyboardInterrupt: 